###  Dense optical flow, specifically the Farneback algorithm from OpenCV, is used on videos of GOES-16 satellite images to estimate cloud speed and direction over the Atmospheric Radiation Measurement (ARM) Southern Great Plains (SGP) Central Facility. 
Optical Flow in OpenCV: https://learnopencv.com/optical-flow-in-opencv/

In [1]:
import cv2
import numpy as np

In [2]:
def dense_optical_flow(method, video_path, params=[], to_gray=True):
    
    # Read the video
    cap = cv2.VideoCapture(video_path)
    
    # Read the first frame of the video
    ret, old_frame = cap.read()

    # Create HSV & make value a constant
    hsv = np.zeros_like(old_frame)
    hsv[..., 1] = 255

    # Convert the BGR image to gray-scale
    if to_gray:
        old_frame = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    
    i=0

    while True:
        
        # Read the next frame
        ret, new_frame = cap.read()
        frame_copy = new_frame
        if not ret:
            break
            
        # Convert the BGR image to gray-scale
        if to_gray:
            new_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
            
        # Calculate Optical Flow
        flow = method(old_frame, new_frame, None, *params)

        # Encoding: convert the algorithm's output into Polar coordinates
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
        # Use Hue and Saturation to encode the Optical Flow
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        
        # Convert HSV image into BGR
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
        # Show video frames and optical flow
        cv2.imshow("frame", frame_copy)
        cv2.imshow("optical flow", bgr)
       
        k = cv2.waitKey(25) & 0xFF
        if k == 27:
            break
        
        # Save optical flow hsv images and video frames
        name = f'./opticalhsv_160_vis{i}.png'
        name2 = f'./frame_160_vis{i}.png'
        cv2.imwrite(name, bgr)
        cv2.imwrite(name2, frame_copy)
        
        # Select the magnitude (speed) and angle at a single pixel
        magn = mag[540, 960]
        angl = ang[540, 960]
        
        # Select the magnitude (speed) and angle for an array of pixels
        #magn = mag[535:545,955:965]
        #angl = ang[535:545,955:965]
        
        # Convert pixel units to m/s and radians to degrees
        m_s = (magn / 300) * 500
        angles = (angl * 180) / np.pi
        
        # Append speed in m/s and direction in degrees
        speed_m_s.append(m_s)
        angl_deg.append(angles)
        print(m_s, angles)
        
        old_frame = new_frame 
        i+=1
    
   # Averages motions per pixel over the length of the video 
    speed = (sum(speed_m_s)) / i
    direction = (sum(angl_deg)) / i
    print('Average Speed:', speed)
    print('Average Direction:', direction)
    
    # Averaging the motion at all the selected pixels
    #print(np.mean(speed)) 
    #print(np.mean(direction))

In [3]:
speed_m_s = []
angl_deg = [] 
method = cv2.calcOpticalFlowFarneback
params = [0.5, 3, 15, 3, 5, 1.2, 0]  # default Farneback algorithm parameters
videopath = 'C:/Users/Margo/GOES16/GOES16_vis_17_19Z_160.mp4'

frames = dense_optical_flow(method, videopath, params, to_gray=True)

1.3614191611607869 68.97659144671971
1.782670219739278 147.07088437875947
5.812459786732991 72.45749490437916
28.546953201293945 95.16434268687065
23.80471070607503 95.70260257460268
12.602213223775228 116.01768886448497
7.411916255950928 84.56003686831899
1.5845641493797302 190.3183034862029
0.8920150995254517 72.28385783529909
2.7705983320871987 175.303785501709
2.2095433870951333 50.128120941462996
7.325488726298015 121.51370986316245
1.1974190672238667 143.96467826877776
1.3837626576423645 20.430572835916152
0.582699328660965 271.5302089441383
0.416504442691803 99.58048345701245
0.1531852533419927 277.01269250788084
3.1057055791219073 95.6158864929002
11.216883659362793 100.93781279982322
1.669541597366333 144.63894088326307
0.6673141320546468 351.4800866887521
0.48353443543116253 81.39439445217981
4.546242554982503 82.24480081517059
17.885637283325195 107.10928486590161
Average Speed: 5.808874260013302
Average Direction: 127.726552598487


In [4]:
cv2.destroyAllWindows()